# Dataset Information Creation
Create a json file to represent the files inside the dataset

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import numpy as np
import json
import torch.nn as nn
import os
from birdlib import utils

In [2]:
# sudo modprobe nvidia_uvm
torch.cuda.is_available()

True

In [3]:
DATASET_NAME = "dataset"
MODEL_NAME = 'VanillaCNN'
DATASET_VAR = 'augm_final_4'

In [4]:
DATASET_PATH = f'../segments/{DATASET_NAME}'
TRAIN_PATH = f"{DATASET_PATH}/train"
VALID_PATH = f"{DATASET_PATH}/valid"
TEST_PATH = f"{DATASET_PATH}/test"
MODEL_PATH = f'./models/{MODEL_NAME}'

# Create Dataset Config

In [5]:
def create_dataset_config(dataset_name, config_file_name='dataset_config.json'):
    saving_path = f"utils/{dataset_name}/{config_file_name}"
    if os.path.exists(saving_path):
        print("Dataset config already created!")
        with open(saving_path) as f:
            return json.load(f)

    mappings = utils.get_mappings(TRAIN_PATH)
    samples = utils.collect_samples(TRAIN_PATH, VALID_PATH, TEST_PATH, mappings)

    dataset_config = {
        "mappings": mappings,
        "samples": samples
    }
    with open(saving_path, "w") as f:
        json.dump(dataset_config, f)
    print("Saved new dataset config")
    return dataset_config

In [6]:
dataset_config = create_dataset_config(DATASET_NAME, f'dataset_config_{DATASET_VAR}.json')
mappings = dataset_config["mappings"]

Dataset config already created!


## Load the model

In [7]:
model_class = utils.load_model_class(MODEL_NAME)
model = model_class(len(mappings))

# Spectograms Creation

In [8]:
SPECS_TRAIN_PATH = f"{DATASET_PATH}/train_specs"
SPECS_VALID_PATH = f"{DATASET_PATH}/valid_specs"
SPECS_TEST_PATH = f"{DATASET_PATH}/test_specs"
os.makedirs(SPECS_TRAIN_PATH, exist_ok=True)
os.makedirs(SPECS_VALID_PATH, exist_ok=True)
os.makedirs(SPECS_TEST_PATH, exist_ok=True)
utils.specs_generation(TRAIN_PATH, SPECS_TRAIN_PATH, dataset_config['mappings'])
utils.specs_generation(VALID_PATH, SPECS_VALID_PATH, dataset_config['mappings'])
utils.specs_generation(TEST_PATH, SPECS_TEST_PATH, dataset_config['mappings'])

Processing: Aeroplane
Processing: Muscicapa striata_Spotted Flycatcher
Processing: Periparus ater_Coal Tit
Processing: Regulus regulus_Goldcrest


Processing: Anthus trivialis_Tree Pipit
Processing: Vegetation
Processing: Troglodytes troglodytes_Eurasian Wren
Processing: Erithacus rubecula_European Robin
Processing: None
Processing: Parus major_Great Tit
Processing: Certhia familiaris_Eurasian Treecreeper
Processing: Phylloscopus collybita_Common Chiffchaff
Processing: Coccothraustes coccothraustes_Hawfinch
Processing: Wind
Processing: Turdus merula_Eurasian Blackbird
Processing: Loxia curvirostra_Common Crossbill
Processing: Regulus ignicapilla_Common Firecrest
Processing: Sylvia atricapilla_Eurasian Blackcap
Processing: Lophophanes cristatus_Crested Tit
Processing: Fringilla coelebs_Common Chaffinch
Processing: Aeroplane
Processing: Muscicapa striata_Spotted Flycatcher
Processing: Periparus ater_Coal Tit
Processing: Regulus regulus_Goldcrest
Processing: Anthus trivialis_Tree Pipit
Processing: Vegetation
Processing: Troglodytes troglodytes_Eurasian Wren
Processing: Erithacus rubecula_European Robin
Processing: None
Processing: P

'✅ Spettrogrammi generati e salvati.'

# Model

In [9]:
def train_model(dataset_config, model, model_name, dataset_var, epochs=10, batch_size=100, lr=1e-5, patience=3, early_stop_patience=10, early_stop_tollerance=1e-5, print_freq=100, load_weights=False, checkpoint_name='checkpoint.pth'):
    history_loss = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training {model_name} on: {device}")

    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=patience, threshold=1e-4
    )
    history_loss = []
    best_loss = float("inf")
    starting_epoch = 0

    saving_path = f'models/{model_name}/{dataset_var}/checkpoint.pth'
    if load_weights:
        if not os.path.exists(saving_path):
            print("No weights found!")
            return None
        checkpoint = torch.load(saving_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        history_loss = checkpoint['history_loss']
        best_loss = checkpoint['avg_loss']
        starting_epoch = len(history_loss)
        epochs += starting_epoch
        print(f"Model Loaded!")
    print(f"Starting from epoch {starting_epoch} for {epochs} epochs")
        
    print("Loading training data...")
    train_loader = utils.get_dataloader(dataset_config, split="train", batch_size=batch_size)
    valid_loader = utils.get_dataloader(dataset_config, split="valid", batch_size=batch_size)
    print("Loaded!")
    
    model.train()
    early_stop_counter = 0
    for epoch in range(epochs):
        epoch += starting_epoch
        model.train()
        running_loss = 0.0
        print(f"\n🎯 Starting epoch {epoch + 1}/{epochs}")

        for batch_index, (mel_spec, labels, _) in enumerate(train_loader):
            mel_spec = mel_spec.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(mel_spec)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch = len(history_loss)

            running_loss += loss.item()
            
            if batch_index % print_freq == 0:
                print('Epoch: [{0}][{1}/{2}], Loss: {loss:.5f}'.format(epoch, batch_index, len(train_loader), loss=loss))

        train_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch + 1} - Loss: {train_loss:.5f}")

        # Validation phase
        print("Validating...")
        model.eval()
        with torch.no_grad():
            valid_loss = 0.0
            for mel_spec, labels, _ in valid_loader:
                mel_spec = mel_spec.to(device)
                labels = labels.to(device)
                outputs = model(mel_spec)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()

            valid_loss /= len(valid_loader)
            print(f"Validation Loss: {valid_loss:.5f}")
        
        # weighted average loss (valid loss * 0.2 + train loss * 0.8)
        avg_loss = 0.8 * (running_loss / len(train_loader)) + 0.2 * valid_loss
        history_loss.append(avg_loss)
        scheduler.step(valid_loss)
        
        np.save(f'models/{model_name}/{dataset_var}/history_loss.npy', history_loss)
        if valid_loss < best_loss - early_stop_tollerance:
            best_loss = valid_loss
            early_stop_counter = 0
            print(f"💾 Saving improved model at epoch {epoch+1} with valid_loss={valid_loss:.5f}")
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'avg_loss': avg_loss,
                'history_loss': history_loss
            }, saving_path)
        else:
            early_stop_counter += 1
            print(f"🛑 No improvement — early stop counter: {early_stop_counter}/{early_stop_patience}")

        print(f"🔁 Epoch {epoch+1} completed - Avg loss: {avg_loss:.7f} - LR: {optimizer.param_groups[0]['lr']:.1e}")

        if early_stop_counter >= early_stop_patience:
            print(f"\n🚨 Early stopping triggered after {early_stop_patience} epochs without improvement.")
            break

    print("✅ Training completed")


    return model


In [ ]:
os.makedirs(f'models/{MODEL_NAME}/{DATASET_VAR}', exist_ok=True)
model = train_model(dataset_config, model, model_name=MODEL_NAME, dataset_var=DATASET_VAR, epochs=200, batch_size=64, lr=1e-5, load_weights=False)

Training VanillaCNN on: cuda


Starting from epoch 0 for 200 epochs
Loading training data...
Loaded!

🎯 Starting epoch 1/200
Epoch: [0][0/379], Loss: 0.69100
Epoch: [0][100/379], Loss: 0.19772
Epoch: [0][200/379], Loss: 0.17350
Epoch: [0][300/379], Loss: 0.21042
Epoch 1 - Loss: 0.23211
Validating...
Validation Loss: 0.20536
💾 Saving improved model at epoch 1 with valid_loss=0.20536
🔁 Epoch 1 completed - Avg loss: 0.2267632 - LR: 1.0e-05

🎯 Starting epoch 2/200
Epoch: [1][0/379], Loss: 0.17832
Epoch: [1][100/379], Loss: 0.18081
Epoch: [1][200/379], Loss: 0.17640
Epoch: [1][300/379], Loss: 0.16574
Epoch 2 - Loss: 0.18044
Validating...
Validation Loss: 0.19159
💾 Saving improved model at epoch 2 with valid_loss=0.19159
🔁 Epoch 2 completed - Avg loss: 0.1826660 - LR: 1.0e-05

🎯 Starting epoch 3/200
Epoch: [2][0/379], Loss: 0.17413
Epoch: [2][100/379], Loss: 0.17291
Epoch: [2][200/379], Loss: 0.16212
Epoch: [2][300/379], Loss: 0.17717
Epoch 3 - Loss: 0.16823
Validating...
Validation Loss: 0.18782
💾 Saving improved model a

In [ ]:
import numpy as np
history_loss = np.load(f'models/{MODEL_NAME}/history_loss_{DATASET_VAR}.npy')
plt.figure(figsize=(8, 5))
plt.plot(history_loss, linestyle='-', color='tomato', label="Loss")
# plt.title(title)
plt.xlabel("Epoch")
# plt.ylabel(ylabel)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'models/VanillaCNN/history_loss_augm_final_4.npy'

In [ ]:
torch.cuda.empty_cache()
os.getpid()

26260